## Tutorial 2: A Complete HoloClean Pipeline

In this tutorial, we will walk step-by-step through the process of repairing a dataset in `HoloClean`. 

The dataset in question contains information about hospitals and is commonly-used for research purposes. Errors are present in ~5% of the cells and there is significant duplicate information - the ideal environment for `HoloClean`.

### Step 1: Data Loading

We begin by instantiating the `HoloClean` and `Session` objects needed to run the repairs. For a more detailed overview of these objects and the rest of our infrastructure, please see our [Data Loading & Denial Constraints Tutorial](Tutorial_1.ipynb).

In [1]:
from holoclean.holoclean import HoloClean, Session

holo = HoloClean(mysql_driver = "../holoclean/lib/mysql-connector-java-5.1.44-bin.jar" )
session = Session(holo)

Next, we load in the data and denial constraints needed for this dataset. Both pieces of information are stored in the MySQL database.

In [2]:
data_path = "/home/milad/Desktop/DFKI/abstraction-layer/tools/Holoclean/HoloClean/tutorial/toy_hospital.csv"

## loads data into our database and returns pyspark dataframe of initial data
data = session.load_data(data_path)

dc_path = "/home/milad/Desktop/DFKI/abstraction-layer/tools/Holoclean/HoloClean/tutorial/data/hospital_constraints (copy).txt"

# loads denial constraints into our database and returns a simple list of dcs as strings 
dcs = session.load_denial_constraints(dc_path)

It's easy to see the dataset has a decent amount of errors. Note the random 'x' characters that have been substituted in.

In [3]:
# all pyspark dataframe commands available
data.select('City','ZipCode','State').show(15)

+----------+-------+-----+
|      City|ZipCode|State|
+----------+-------+-----+
|birmingham|  35233|   al|
|birmingham|  35233|   al|
|birmingham|  35233|   al|
|birminghxm|  35233|   al|
|birmingham|  35233|   al|
|birmingham|  35233|   al|
|birmingham|  35233|   al|
|birmingxam|  35233|   al|
+----------+-------+-----+



In [4]:
# a simple list of strings
dcs

['t1&t2&EQ(t1.ZipCode,t2.ZipCode)&IQ(t1.City,t2.City)']

### Step 2: Error Detection

HoloClean is a supervised error repairing system. In contrast to traditional supervision, we do not ask users to label individual data cells but rely on more high-level supervision signals. These signals are denial constraints or other, custom-made error detectors that split the data into two categories, "clean" and "don't-know". Using that split, our later steps of the process will be able to learn the features of a "clean" cell and perform inference on the values of the "don't-know" cells.

Please see our <a href=http://pages.cs.wisc.edu/~thodrek/blog/holoclean.html>blog post</a> for more information

In this tutorial, we will use HoloClean's built in error detector that uses denial constraints to perform this split. Any cell that participates in a violation of a DC is marked "don't-know", the rest are treated as clean. If you wish to develop a custom error detector, please see our [Error Detectors Tutorial](Tutorial_3.ipynb) for a walkthrough.

In [5]:
from holoclean.errordetection.mysql_dcerrordetector import Mysql_DCErrorDetection

# instantiate Holoclean's built in error detector
detector = Mysql_DCErrorDetection(session.Denial_constraints, holo, session.dataset)

# both clean and dirty sets are returned as pyspark dataframes
clean, dirty = session.detect_errors(detector)

OperationalError: (_mysql_exceptions.OperationalError) (1054, "Unknown column 'table1.index' in 'field list'") [SQL: "INSERT INTO 75516078113_C_dk_temp ( SELECT DISTINCT table1.index as ind, 'ZipCode' AS attr  FROM  75516078113_Init as table1, 75516078113_Init as  table2 WHERE table1.index != table2.index  AND table1.ZipCode=table2.ZipCode AND table1.City<>table2.City );"] (Background on this error at: http://sqlalche.me/e/e3q8)

In [6]:
clean.head(5)

[Row(ind=2, attr=u'state'),
 Row(ind=3, attr=u'state'),
 Row(ind=1, attr=u'state'),
 Row(ind=5, attr=u'city'),
 Row(ind=5, attr=u'zip')]

In [8]:
dirty.head(20)

[Row(ind=1, attr=u'city'),
 Row(ind=1, attr=u'zip'),
 Row(ind=4, attr=u'city'),
 Row(ind=4, attr=u'zip'),
 Row(ind=3, attr=u'zip'),
 Row(ind=2, attr=u'zip'),
 Row(ind=3, attr=u'city'),
 Row(ind=2, attr=u'city')]

### Step 3: Repairing

With the "clean" and "don't-know" split defined, we are ready to perform repairs.

Denial Constraints are the driving force behind this process. Denial constraints are used as features in a softmax regression model. The clean cells are used as training examples to learn the parameters (weights) of this model. Once those weights are defined, we use this model to perform inference on the "don't-know" cells and insert the most likely value for each cell.

This tutorial will simply use the default parameters for our softmax model. Customization of parameters like learning rate, batch size, and number of epochs is described in Tutorial 4 (in development) and is recommended for performance-critical applications.

In [9]:
repaired = session.repair()

100%|██████████| 100/100 [00:00<00:00, 571.35it/s]


As we can see, our repaired dataset has effectively removed large numbers of the 'x' characters

In [8]:
#x=repaired.toPandas()
#x.to_csv("repaired_hospital2.csv", sep=",", header=True, index=False, encoding="utf-8")
#repaired.write.csv('mycsv.csv')
repaired.sort('index')
repaired.write.format('com.databricks.spark.csv').option("header", 'true').save('mycsv3.csv')

In [10]:
repaired = repaired.withColumn("index", repaired["index"].cast("int"))
repaired.sort('index').select('city','zip','state').show(15)

+---------+-----+-----+
|     city|  zip|state|
+---------+-----+-----+
| PEARLAND|77584|   TX|
| PEARLAND|77584|   TX|
| PEARLAND|77584|   TX|
| PEARLAND|77584|   TX|
|PEARLANxD|77584|   TX|
+---------+-----+-----+



### Performance Evaluation

Since this is a research dataset, a clean version is available for us to compare our results to. 

In [8]:
session.compare_to_truth("data/address_10_ground_truth.csv")

AnalysisException: u"Except can only be performed on tables with the same number of columns, but the first table has 3 columns and the second table has 13 columns;;\n'Except\n:- AnalysisBarrier\n:     +- Project [tid#693, attr_name#691, attr_val#692]\n:        +- Relation[probability#689,vid#690,attr_name#691,attr_val#692,tid#693,domain_id#694] JDBCRelation((Select * from 8385910373_Inferred_values) as tablename) [numPartitions=1]\n+- AnalysisBarrier\n      +- Relation[index#729,RecID#730,FirstName#731,MiddleName#732,LastName#733,Address#734,City#735,State#736,ZIP#737,POBox#738,POCityStateZip#739,SSN#740,DOB#741] csv\n"